<h1>How to chose a perfect spot for a new dental office in Toronto, Canada</h1>

<h2>Problem</h2>
<p>
    Imagine you are a young aspiring dentist. You studied and worked hard and now ready to go into business on your own. It is a big decision and very expensive too. The bank is willing to lend you money but wants to make sure that you can earn enough to cover the loan payments. 

How do you decide on the area? There are a lot of dentists out there, in every neighbourhood. How would you even compare one neighbourhood to the other? These are the questions that I will try to tackle in this project.

I'm starting with a hypothesis that it is possible to asses and rank neighbourhoods based on the level of competition. A single dentist can take care of a finite number of patients. Therefore we can calculate the maximum number of dentists for each neighbourhood and see how this number compares to the actual number of clinics.
    </p>

<h2>Data I will use</h2>

<b>List of dentist's offices by neighbourhood: </b>
    - Foursquare and a web directory (https://www.toronto.com/search/directory/?q=dentist&location=toronto&city=northyork-on) I will need to scrape the information from the directory and group it based on postal codes.
    
<b>Total population by neighbourhood: </b>
    - This information is available on the Stats Canada website. https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Table.cfm?Lang=Eng&T=1201&SR=1&S=22&O=A&RPP=9999&PR=0